In [46]:
import torch
import torchvision as tv
import glob
import os
import sys
import numpy as np
import pandas as pd
import PIL
import time

In [47]:
class MountainDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir="./datasets/test", transforms=None):
        super(MountainDataset, self).__init__()
        if not os.path.isdir(root_dir):
            raise FileNotFoundError("Dir {} Error".format(root_dir))
        
        self._data = glob.glob(os.path.join(root_dir, "*.png"))
        self._root = root_dir
        if transforms:
            self._transforms = transforms
        else:
            self._transforms = tv.transforms.ToTensor()
    
    def __getitem__(self, index):
        image = PIL.Image.open(self._data[index])
        data = self._transforms(image)
        label = self._data[index]
        
        return (data, label)
    
    def __len__(self):
        return len(self._data)

In [48]:
model = tv.models.inception_v3()
model.fc = torch.nn.Linear(2048, 2) # change output layer
model.cuda()
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
model.to(device)
criterion = torch.nn.modules.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [49]:
model.load_state_dict(torch.load("checkpoints/latest.pth"), strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [63]:
transform_testing = tv.transforms.Compose([
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])

dataset_testing = MountainDataset(transforms=transform_testing)
data_loader_testing = torch.utils.data.DataLoader(dataset_testing, batch_size = 20, shuffle=True, num_workers=3)

In [66]:
sm = torch.nn.Softmax()
for (inputs, labels) in data_loader_testing:
    for file in labels:
        print(file)
    inputs = inputs.to(device)
    labels = inputs.to(device)
    
    optimizer.zero_grad()
    
    outputs = model(inputs)
    
    pro = sm(outputs[:, 1])
    print(outputs)
    print(pro)
    print()

./datasets/test/perlin_022_fake_B.png
./datasets/test/perlin_057_fake_B.png
./datasets/test/perlin_002_4oct.png
./datasets/test/100,100_Taiwan.png
./datasets/test/perlin_058_fake_B.png
./datasets/test/perlin_003_4oct.png
./datasets/test/perlin_006_4oct.png
./datasets/test/perlin_005_4oct.png
./datasets/test/perlin_021_fake_B.png
./datasets/test/perlin_004_4oct.png
./datasets/test/perlin_056_fake_B.png
./datasets/test/perlin_007_8oct.png
./datasets/test/perlin_024_fake_B.png
./datasets/test/perlin_023_fake_B.png
./datasets/test/perlin_059_fake_B.png
tensor([[-1.7771,  2.3334],
        [ 0.5738,  0.0993],
        [ 1.5491, -0.8460],
        [-3.7806,  4.0441],
        [ 0.0931,  0.6074],
        [ 2.6711, -2.4182],
        [ 1.3363, -0.7270],
        [ 1.8782, -1.3025],
        [-0.2240,  1.1092],
        [ 2.1039, -1.4347],
        [ 0.2330,  0.2084],
        [ 2.5624, -1.9204],
        [-0.4495,  1.2247],
        [-0.3751,  1.0232],
        [-0.7357,  1.6136]], device='cuda:0', grad_fn

/root/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
